In [114]:
# Check if GPU is available
if tf.test.is_gpu_available():
    print("GPU is available.")
    # Shift operations to GPU by creating and training your model within this block.
    # For example, you can define your model here:
    # model = YourModel()
else:
    print("GPU is not available.")

GPU is not available.


In [115]:
!pip install transformers

In [116]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import gzip
import json
import re
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification


In [118]:


file_path = '/content/Appliances_5.json.gz'

# Open the compressed .gz file
with gzip.open(file_path, 'rt') as f:
    data = []

    # Read and process each line in the file
    for line in f:
        try:
            # Parse each line as JSON
            data_dict = json.loads(line)

            # Append the parsed data to the list
            data.append(data_dict)
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON on line: {line}")

In [119]:
# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

In [120]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"08 22, 2013",A34A1UP40713F8,B00009W3I4,{'Style:': ' Dryer Vent'},James. Backus,I like this as a vent as well as something tha...,Great product,1377129600,NaN,NaN
1,5.0,True,"02 8, 2016",A1AHW6I678O6F2,B00009W3PA,{'Size:': ' 6-Foot'},kevin.,good item,Five Stars,1454889600,NaN,NaN
2,5.0,True,"08 5, 2015",A8R48NKTGCJDQ,B00009W3PA,{'Size:': ' 6-Foot'},CDBrannom,Fit my new LG dryer perfectly.,Five Stars,1438732800,NaN,NaN
3,5.0,True,"04 24, 2015",AR3OHHHW01A8E,B00009W3PA,{'Size:': ' 6-Foot'},Calvin E Reames,Good value for electric dryers,Perfect size,1429833600,NaN,NaN
4,5.0,True,"03 21, 2015",A2CIEGHZ7L1WWR,B00009W3PA,{'Size:': ' 6-Foot'},albert j. kong,Price and delivery was excellent.,Five Stars,1426896000,NaN,NaN


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277 entries, 0 to 2276
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         2277 non-null   float64
 1   verified        2277 non-null   bool   
 2   reviewTime      2277 non-null   object 
 3   reviewerID      2277 non-null   object 
 4   asin            2277 non-null   object 
 5   style           38 non-null     object 
 6   reviewerName    2277 non-null   object 
 7   reviewText      2277 non-null   object 
 8   summary         2277 non-null   object 
 9   unixReviewTime  2277 non-null   int64  
 10  vote            2074 non-null   object 
 11  image           828 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 198.0+ KB


In [122]:
df["Sentiment"] = df["overall"].apply(lambda score: "positive" if score >= 3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})

In [123]:
df['short_review'] = df['summary']  #.str.decode("utf-8")
df = df[["short_review", "Sentiment"]]

In [124]:
df.isnull().sum()

short_review    0
Sentiment       0
dtype: int64

In [125]:
df = df.dropna()

# Reset the index
df = df.reset_index(drop=True)

In [126]:
df['short_review'] = df['short_review'].str.replace(r'[^\w\s]', '', regex=True)

In [127]:
df["Sentiment"].value_counts()

1    2255
0      22
Name: Sentiment, dtype: int64

In [128]:
# # Assuming your original dataset is in the variable 'df'
# # Sample 2,500 positive samples
# positive_samples = df[df['Sentiment'] == 1].sample(4000)

# # Sample 2,500 negative samples
# negative_samples = df[df['Sentiment'] == 0].sample(2219)

# # Combine the sampled dataframes
# balanced_df = pd.concat([positive_samples, negative_samples])

# # Shuffle the dataset to mix the positive and negative samples
# df = balanced_df.sample(frac=1).reset_index(drop=True)

In [129]:
df.tail(7)

,short_review,Sentiment
2270,Works Great Just Use Common Sense When Doing It,1
2271,good kit with some caveats,1
2272,Worked like a charm,1
2273,Excellent product and showed me that my vent o...,1
2274,Five Stars,1
2275,Five Stars,1
2276,Five Stars,1


In [130]:
reviews = df['short_review'].values.tolist()
labels = df['Sentiment'].tolist()


In [131]:
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [132]:
len( validation_labels)

456

In [133]:
#Assign tokenizer object to the tokenizer class
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [134]:
tokenizer([training_sentences[0]], truncation=True,padding='max_length', max_length=30, return_tensors="tf" )

{'input_ids': <tf.Tensor: shape=(1, 30), dtype=int32, numpy=
array([[ 101, 2515, 2054, 2009, 3791, 2000,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 30), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}

In [135]:
train_encodings = tokenizer(training_sentences,truncation=True,padding=True)
val_encodings = tokenizer(validation_sentences,truncation=True,padding=True)


In [136]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),training_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings),validation_labels))

In [137]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [138]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/2
 32/114 [=======>......................] - ETA: 5:50 - loss: 0.6564 - accuracy: 0.4629

KeyboardInterrupt: ignored

In [ ]:
model.save_pretrained("./sentiment")

In [ ]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("./sentiment")

In [ ]:
test_sentence = "This is a really good product. I love it"


predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

tf_output = loaded_model.predict(predict_input)[0]


tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])

In [ ]:
# dataset has highly imbalanced has 22 of negative reviews rest evrything is positive . we can try dataaugmentation with differet methods oversampling generating artficial reviews.
#also use software review datset and take only 3000 of positive 2000 of negative
test_sentence = "worst product. Not worth it."


predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

tf_output = loaded_model.predict(predict_input)[0]


tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])